In [1]:
import json
import pandas as pd
from pandas import json_normalize
from datasets import load_from_disk
import warnings
warnings.filterwarnings('ignore')

# Reader 데이터 경로

In [2]:
dataset = load_from_disk("/opt/ml/input/data/train_dataset")
test_dataset=load_from_disk("/opt/ml/input/data/test_dataset")

train_dataset = dataset["train"]
valid_dataset = dataset["validation"]
test_dataset=test_dataset['validation']

train_df=pd.DataFrame(train_dataset)
valid_df=pd.DataFrame(valid_dataset)
test_df=pd.DataFrame(test_dataset)

# Rtrieval 데이터 경로

In [3]:
file_path = "/opt/ml/input/data/wikipedia_documents.json"

with open(file_path, "r") as json_file:
    json_data = json.load(json_file)

# SOTA , 내 예측 경로

In [4]:
### 비교할 json파일의 경로를 넣어주세요!
# sota_path = "./outputs/sota_0509.json"
sota_path = "./outputs/twice_topk_40_filter1/predictions.json"
my_pre_path="./outputs/twice_topk_40_filter1_setting2/predictions.json"

with open(sota_path, "r") as json_file:
    sota= json.load(json_file)
with open(my_pre_path, "r") as json_file:
    my_pre= json.load(json_file)

# test_df 비교

In [5]:
sota_list=list(sota.values())
my_pre_list=list(my_pre.values())

diff=[]
for js in sota_list:
    if js not in my_pre_list:
        diff.append(sota_list.index(js))
        
sota_diff_list=[]
my_pre_diff_list=[]

for df in diff:
    sota_diff_list.append(sota_list[df])
    my_pre_diff_list.append(my_pre_list[df])

sota_diff=test_df.iloc[diff]
sota_diff["SOTA_ans"]=sota_diff_list
sota_diff["my_ans"]=my_pre_diff_list
sota_diff["SOTA_ans_len"]=list(map(len,sota_diff_list))
sota_diff["my_ans_len"]=list(map(len,my_pre_diff_list))

sota_diff=sota_diff.drop(['id'],axis=1)

In [6]:
print("#############################################################")
print(f"SOTA와 비교하여 다른 답변의 개수는 {len(sota_diff)}개 입니다.")
print("#############################################################")
sota_diff.sort_values(by=["SOTA_ans_len","my_ans_len"],ascending=False).head(50)

#############################################################
SOTA와 비교하여 다른 답변의 개수는 1개 입니다.
#############################################################


,question,SOTA_ans,my_ans,SOTA_ans_len,my_ans_len
311,navigation'이 올라온 사이트는?,보배드림 사이트,본인 사운드클라우드 계정,8,13


# 전체

In [15]:
test_df["SOTA_ans"]=sota_list
test_df["my_ans"]=my_pre_list
test_df.sample(10)

,question,id,SOTA_ans,my_ans
197,매우 센 중력에 의해 빛이 탈출하지 못하는 천체는?,mrc-0-004709,블랙홀,블랙홀
236,히키코모리가 유독 많아졌던 세대는?,mrc-0-002518,빙하기 세대,빙하기 세대
379,프랑크 왕족들의 무덤이 있는 곳은?,mrc-0-000668,‘논수골’,‘논수골’
497,조선인민공화국에서 최고로 높은 직위에 오른 자는?,mrc-0-000394,알리 하메네이,장성택
163,멕시코 연방 정부는 메탈클래드에게 얼마를 보상하라고 하였나?,mrc-0-000065,"1,668만 5천 달러","1,668만 5천 달러"
31,이타미 역이 종착역이었을 때 존재했지만 지진 이후 없어진 것은?,mrc-0-004646,건늠선,건늠선
2,돌푸스에게 불특정 기간동안 하원이 잠시 쉬는 것을 건의 받았던 인물은?,mrc-0-002191,대통령인 빌헬름 미클라스,대통령인 빌헬름 미클라스
183,린후이인이 원래 진학하고 싶어했던 과는?,mrc-0-001384,건축학과,건축학과
21,초등학생 일기검사의 인권침해 여부를 판단하는 주체는?,mrc-0-003683,국가인권위원회,국가인권위원회
470,이론물리학에 비해 수학적인 부분을 더 강조하는 학문은?,mrc-1-001729,수리물리학,수리물리학
